# Anwendungsbeispiel: Nachrichten-Crawler

In diesem Beispiel wollen wir Nachrichten von der Webseite des Deutschlandfunk abrufen und indizieren.

In [ ]:
import re

INDEX = dict()

def index(url, doc):
    
    for e in doc.xpath("//article[@class='dlf-articledetail']//div[@class='text']"):             
        for word in re.sub("[^\w]+", " ", e.text_content()).split():
            if word in INDEX:
                INDEX[word].add(url)
            else:
                INDEX[word] = { url }
                        

In [ ]:
import requests 
import lxml.html as lh
from urllib.parse import urljoin

base = 'https://deutschlandfunk.de'
articles = set()

resp = requests.get(base)
if resp:
    doc = lh.fromstring(resp.content)
    for link in doc.xpath("//article//a"):
        href = urljoin(base, link.get('href'))
        if not href.startswith(base): continue
        articles.add(href)
        
for url in articles:
    print(f"crawling {url}")
    resp = requests.get(url)
    if resp:
        doc = lh.fromstring(resp.content)
        index(url, doc)

In [ ]:
def search(phrase):
    hits = set()
    for word in (phrase.split() & INDEX.keys()):
        hits |= INDEX[word]
    
    return hits

In [ ]:
search('Brexit') 